In [96]:
from selenium import webdriver
import time
from collections import defaultdict
import pprint
import re
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import logging
import math
import csv
from titlecase import titlecase

In [76]:
def scroll(driver, timeout, allSongList):
    scroll_pause_time = timeout

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    i = 0
    while i < 10:

        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(scroll_pause_time)

        allSongs = driver.find_element_by_xpath('//*[@id="Web.TemplatesInterface.v1_0.Touch.GalleryTemplateInterface.GalleryTemplate_1"]/music-container/div[2]/div/div')
        templist = allSongs.find_elements_by_xpath(".//*")
        for song in templist:
            songtoAdd = []
            songtoAdd.append(song.get_attribute("primary-text"))
            songtoAdd.append(song.get_attribute("secondary-text"))
            allSongList.append(songtoAdd)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            # If heights are the same it will exit the function
            break
        last_height = new_height
        i =  i + 1
    return allSongList

In [77]:
driver = webdriver.Chrome()

driver.get("https://music.amazon.com/forceSignIn?useHorizonte=true")

username = driver.find_element_by_name("email")
username.send_keys("deirdre_w@yahoo.com")
password = driver.find_element_by_name("password")
password.send_keys("Jordanappa1")
driver.find_element_by_id("signInSubmit").click()
time.sleep(5)
driver.get("https://music.amazon.com/recently/played/songs")
allSongList = []
allSongList = scroll(driver, 3, allSongList)

In [78]:
client_id = "6f51739589894b089686876c420ec812"
client_secret = "fce52ef8f7a44305893701b1c01cd549"

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

with open('yourlog.log', 'w'):
    pass

logging.basicConfig(filename='scraper.log', filemode='w', format='%(name)s - %(levelname)s - %(message)s')

cndnsdASL = []
for song in allSongList:
    if song[0] != None:
        cndnsdASL.append(song)

for song in cndnsdASL:
    song[0] = re.sub("[\(\[].*?[\)\]]", "", song[0]).strip()
    song[1] = re.sub("[\(\[].*?[\)\]]", "", song[1]).strip()

for song in cndnsdASL:
    query = "artist:" + song[1] + " track:" + song[0]
    qRes = sp.search(q=query, type="track", limit=1)
    try:
        song.append(qRes['tracks']['items'][0]['duration_ms'])
        print("Duration found for song " + qRes['tracks']['items'][0]['name'])
    except:
        song.append(0)
        logging.error("Spotify API Duration Fetch Error for Song " + song[0])
    try:
        song.append(sp.artist(qRes['tracks']['items'][0]['album']['artists'][0]['id'])['genres'])
        print("Genre found for song " + qRes['tracks']['items'][0]['name'])
    except:
        song.append([])
        logging.error("Spotify API Genre Fetch Error for Song " + song[0])


mical Infatuation
Genre found for song Chemical Infatuation
Duration found for song They Say
Genre found for song They Say
Duration found for song Just A Beautiful Day
Genre found for song Just A Beautiful Day
Duration found for song Electric Halo
Genre found for song Electric Halo
Duration found for song Love In Ruins
Genre found for song Love In Ruins
Duration found for song Reach Down (feat. GNR guitarist Ron Thal)
Genre found for song Reach Down (feat. GNR guitarist Ron Thal)
Duration found for song Down And Out
Genre found for song Down And Out
Duration found for song Between the Lines
Genre found for song Between the Lines
Duration found for song Ordinary God
Genre found for song Ordinary God
Duration found for song Die Trying
Genre found for song Die Trying
Duration found for song Woo Hah!! Got You All in Check
Genre found for song Woo Hah!! Got You All in Check
Duration found for song Heavy Is the Head
Genre found for song Heavy Is the Head
Duration found for song Cortez and Pi

In [97]:

bandDict = defaultdict(list)
for song in cndnsdASL:
    bandDict[song[1]].append(song[0])
bandDurDict = defaultdict(list)
for song in cndnsdASL:
    bandDurDict[song[1]].append(song[2])

genreLOL = []
for song in cndnsdASL:
    genreLOL.append(song[3])
allGenres = [genre for genreList in genreLOL for genre in genreList]
genreCount = defaultdict(list)
for genre in allGenres:
    if genre in genreCount:
        genreCount[genre] += 1
    else:
        genreCount[genre] = 1
sortedGenreCount = sorted(genreCount.items(), key=lambda x: x[1], reverse=1)

bandCount = {band: len(songs) for band, songs in bandDict.items()}
bandDurCount = {band: math.trunc(float(sum(dur)/60000)*100)/100 for band, dur in bandDurDict.items()}

sortedBandCount = sorted(bandCount.items(), key=lambda x: x[1], reverse=1)
sortedBandDurCount = sorted(bandDurCount.items(), key=lambda x: x[1], reverse=1)

songList = [i[0] for i in cndnsdASL]
songCount = {song:songList.count(song) for song in songList}
sortedSongCount = sorted(songCount.items(), key=lambda x: x[1], reverse=1)

totalMinListened = sum(n for _, n in sortedGenreCount)
topGenre = sortedGenreCount[0][0]
print("Total Minutes Listened: " + str(totalMinListened))
print("Top Genre: " + titlecase(topGenre))


Total Minutes Listened: 1735
Top Genre: Alternative Metal


In [86]:
with open("Band_Time_Listened.csv", mode='w', newline='') as writefile:
    writer = csv.writer(writefile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['Band', 'Time Listened (min)'])
    for bands in sortedBandDurCount:
        writer.writerow([bands[0], bands[1]])

with open("Songs_Most_Listened.csv", mode='w', newline='') as writefile:
    writer = csv.writer(writefile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['Song', 'Number of Times Listened'])
    for songs in sortedSongCount:
        writer.writerow([songs[0], songs[1]])